In [ ]:
import pandas as pd 
import xgboost as xgb
import numpy as np
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from bayes_opt.bayes_logistic import bayes_logistic
from sklearn.tree import DecisionTreeClassifier
from bayes_opt.bayes_tree import bayes_tree
from bayes_opt.bayes_xgboost import bayes_xgboost
from bayes_opt.utils import test_learning


train_sets = pickle.load(open("datasets/train_sets_bank.dat","rb"))
test_sets = pickle.load(open("datasets/test_sets_bank.dat","rb"))
val_sets = [pickle.load(open("datasets/val_sets_bank.dat","rb")), pickle.load(open("datasets/val_test_sets_bank.dat","rb"))]
features = pickle.load(open("datasets/bank_features","rb"))
returns = pickle.load(open("datasets/bank_returns","rb"))
outcomes = pickle.load(open("datasets/bank_outcomes","rb"))
rmax = np.zeros(len(returns))
for i in range(len(returns)):
    rmax[i] = returns[i,np.argmax(returns[i])]
    
replication = 5
n_splits = 10
n_steps_xgb = 20
n_trials = 50
results = []

for rep in range(replication):

    bank_ml_results = np.zeros((n_splits, 10))

    for i in range(n_splits):

        config = {
            "cval": True,
            "rep" : rep,
            "set" : i,
            "n_trials" : n_trials,
            "val_sets": val_sets,
            "n_steps": n_steps_xgb
            }    
            
        scaler = MinMaxScaler()
        x_train_scaled = scaler.fit_transform(features[train_sets[rep][i]])
        x_test_scaled = scaler.transform(features[test_sets[rep][i]])            
            
        bayes = bayes_logistic(config, x_train_scaled, outcomes[train_sets[rep][i]], returns[train_sets[rep][i]])
        best_params_logistic = bayes.bayes()
        c, penalty = best_params_logistic.get("c", ""), best_params_logistic.get("penalty", "")
        clf = (LogisticRegression(C = c, penalty=penalty, solver='saga') if penalty != "elasticnet"
               else LogisticRegression(C = c, penalty=penalty, solver='saga', l1_ratio = best_params_logistic.get("l1", "")))          
        clf_fit = clf.fit(x_train_scaled, outcomes[train_sets[rep][i]])
        probs = clf_fit.predict_proba(x_test_scaled)
        test_return, test_outcome = test_learning(probs, returns[test_sets[rep][i]])
        bank_ml_results[i, 0] = test_return / sum(rmax[test_sets[rep][i]])
        bank_ml_results[i, 3] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
        bank_ml_results[i, 4] = f1_score(outcomes[test_sets[rep][i]], test_outcome)
            
        bayes = bayes_tree(config, features[train_sets[rep][i]], outcomes[train_sets[rep][i]], returns[train_sets[rep][i]])
        best_params_tree = bayes.bayes()
        depth, min_samples, cp = best_params_tree.get("max_depth", ""), best_params_tree.get("min_samples_leaf", ""), best_params_tree.get("ccp_alpha", "")
        dt = DecisionTreeClassifier(min_samples_leaf = min_samples, ccp_alpha = cp, max_depth = depth).fit(features[train_sets[rep][i]], outcomes[train_sets[rep][i]])  
        probs = dt.predict_proba(features[test_sets[rep][i]])
        test_return, test_outcome = test_learning(probs, returns[test_sets[rep][i]])
        bank_ml_results[i, 1] = test_return / sum(rmax[test_sets[rep][i]])
        bank_ml_results[i, 5] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
        bank_ml_results[i, 6] = f1_score(outcomes[test_sets[rep][i]], test_outcome)      
            
        bayes = bayes_xgboost(config, features[train_sets[rep][i]], outcomes[train_sets[rep][i]], returns[train_sets[rep][i]])
        best_params_xgb = bayes.bayes()            
        param = {'eta' : best_params_xgb.get("eta", ""), 
                 'max_depth' : best_params_xgb.get("max_depth", ""),
                 'min_child_weight' : best_params_xgb.get("min_child_weight", ""),
                 'gamma' : best_params_xgb.get("gamma", ""),
                 'colsample_bytree' : best_params_xgb.get("colsample_bytree", ""),
                 'objective': 'multi:softprob',
                 'num_class': 2 }                 
        model = xgb.train(param, xgb.DMatrix(features[train_sets[rep][i]], label=outcomes[train_sets[rep][i]]), config["n_steps"])                      
        probs = model.predict(xgb.DMatrix(features[test_sets[rep][i]], label=outcomes[test_sets[rep][i]]))
        test_return, test_outcome = test_learning(probs, returns[test_sets[rep][i]])
        bank_ml_results[i, 2] = test_return / sum(rmax[test_sets[rep][i]])
        bank_ml_results[i, 7] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
        bank_ml_results[i, 8] = f1_score(outcomes[test_sets[rep][i]], test_outcome)   
    
    results.append(bank_ml_results)
            

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import optuna
import torch
import numpy as np

from Models.linearnet import LinearNet
from Models.opt_torch import Optimization
from bayes_opt.utils import give_set, give_score, test_learning

class cslr_bayes: 

    def __init__(self, data, returns, config):
        
        self.device = config.get("device", "cpu")
        self.cval = config.get("cval", False)
        self.n_val_splits = config.get("n_val_splits", 10)    
        self.rep = config.get("rep", 0) 
        self.set_num = config.get("set", 0) 
        self.n_trials = config.get("n_trials", 2)
        self.val_sets = config.get("val_sets", None)
        self.batchnorm = config.get("batchnorm", False)
        self.n_epochs = config.get("n_epochs", 2000)
        self.n_steps = config.get("n_steps", 1000)        
        self.batches = config.get("batches", None)
        self.batch_size = config.get("batch_size", 1000)
        self.max_batch = config["max_batch"]
        self.layers = config.get("layers", None)        
        self.numlayer = config.get("dnn_layers", 1)    
        self.hidden_size = config.get("hidden_size", [])
        self.lr = config.get("lr", None)   
        self.lr_rate = config.get("lr_rate", 0.001)    
        self.data = data
        self.returns = returns
    
    def train_bayes_linear(self, trial):                 
                  
        space = {'lr_rate' : (trial.suggest_uniform('lr_rate', 0.00005, 0.01) if self.lr is not None else self.lr_rate)
                'batch_size': (trial.suggest_int('batch_size', self.batches[0], self.batches[1]) if self.batches is not None
                               else self.batch_size),
                 'numlayer': (trial.suggest_int('layers', self.layers[0], self.layers[1]) if self.layers is not None
                              else self.numlayer)
                }
              
        config_nn = {
            "n_inputs" : self.data.shape[1],
            "dnn_layers" : space['numlayer'],
            "hidden_size" : self.hidden_size,
            "n_outputs" : self.returns.shape[1],
            "batchnorm" : self.batchnorm,
            "n_epochs": self.n_epochs,
            "n_steps": self.n_steps
        }
        
        if self.cval == False:
              
            model = LinearNet(config_nn).to(self.device)
            config_nn["batch_size"] = (2**space['batch_size'] if space['batch_size']<self.max_batch else len(self.data))                                                
            optimizer = torch.optim.Adam(model.parameters(), lr=space['lr_rate'])
            optimization = Optimization(model, optimizer, config_nn)
            optimization.train(self.data, self.returns)
            _, _, test_probs = optimization.evaluate(self.data, self.returns)
            test_return = test_learning(test_probs, returns)
            
        else:
            
            test_return = np.zeros(self.n_val_splits)
            
            if self.val_sets is None:
                skf = KFold(self.n_val_splits, shuffle=True)
                for train_index, test_index in skf.split(self.data):
                    x_train, x_test = self.data[train_index], self.data[test_index]
                    r_train, r_test = self.returns[train_index], self.returns[test_index]
                    model = LinearNet(config_nn).to(self.device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=space['lr_rate'])
                    config_nn["batch_size"] = (2**space['batch_size'] if space['batch_size']<self.max_batch else len(x_train))                                    
                    optimization = Optimization(model, optimizer, config_nn)
                    optimization.train(x_train, r_train, x_test, r_test)
                    _, _, test_probs = optimization.evaluate(x_test, r_test)
                    test_return[i],_ = test_learning(test_probs, r_test)            
            
            else:
                for i in range(self.n_val_splits):
                    train_index, test_index = give_set(self.rep, self.set_num, i, self.val_sets)
                    x_train, x_test = self.data[train_index], self.data[test_index]
                    r_train, r_test = self.returns[train_index], self.returns[test_index]

                    model = LinearNet(config_nn).to(self.device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=space['lr_rate'])
                    config_nn["batch_size"] = (2**space['batch_size'] if space['batch_size']<self.max_batch else len(x_train))                
                    optimization = Optimization(model, optimizer, config_nn)
                    optimization.train(x_train, r_train, x_test, r_test)
                    _, _, test_probs = optimization.evaluate(x_test, r_test)
                    test_return[i],_ = test_learning(test_probs, r_test)
        
        return test_return.mean()

    def bayes(self):

        sampler = optuna.samplers.TPESampler()    
        study = optuna.create_study(sampler=sampler, direction='maximize')
        study.optimize(func=self.train_bayes_linear, n_trials=self.n_trials)
    
        return study.best_params



In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import torch
import numpy as np

from bayes_opt.bayes_linearnet import cslr_bayes
from Models.linearnet import LinearNet
from Models.opt_torch import Optimization
from Models.gurobi_opt import Optimization_MIP
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from gurobipy import *


class cslr: 

    def __init__(self, config, x_train, r_train, x_test, r_test, x_val=None, r_val=None):
        self.device = config.get("device", "cpu")
        self.n_epochs = config.get("n_epochs", 1000)
        self.n_steps = config.get("n_steps", "500")    
        self.lr_rate = config.get("lr_rate", 5e-3)   
        self.hidden_size = config.get("hidden_size", 0)
        self.batch_size = config.get("batch_size", 1000)
        self.num_layer = config.get("numlayer", 1)
        self.batchnorm = config.get("batchnorm", False)
        self.scaler = config.get("scaler", None)
        self.batches_bayes = config.get("batches", None)
        self.lr_bayes = config.get("lr", None)
        self.layers_bayes = config.get("lr", None)
        self.bayes = config.get("bayes", False)
        self.rep = config.get("rep", None)
        self.set = config.get("set", None)
        self.cval = config.get("cval", None)
        self.n_val_splits = config.get("n_val_splits", 10)
        self.val_sets = config.get("val_sets", None)
        self.n_trials = config.get("n_trials", None)   
        self.time_limit = config.get("time_limit", 100.0)
        self.give_initial = config.get("initial", False)        
        self.beta_initial = None
        self.x_train = x_train
        self.x_test = x_test
        self.r_train = r_train
        self.r_test = r_test
        self.x_val = x_val
        self.r_val = r_val
        self.max_batch = config.get("max_batch", len(self.x_train))
        self.num_class = r_train.shape[1]
        self.num_features = x_train.shape[1]
        self.model = None
        
    def opt_initial(self, beta_opt):
        
        scores = np.zeros((len(self.x_train), self.num_class))
        scores_diff = np.zeros((len(self.x_train), ((self.num_class * (self.num_class - 1)) // 2)))
        for i in range(len(self.x_train)):
            for k in range(self.num_class):
                scores[i,k] = sum(self.x_train[i,j] * beta_opt[k,j].item() for j in range(self.num_features))
        diff_ind = 0
        for k in range(self.num_class-1):
            for t in range(self.num_class-(k+1)):
                scores_diff[:,diff_ind] = np.subtract(scores[:,k], scores[:,(k+t+1)])
                diff_ind += 1
        is_deviate = []
        for i in range(len(scores_diff)):
            if (abs(scores_diff[i,:]) < 0.01).any() or (scores[i,:] > 100).any():
                is_deviate.append(i)
        return  np.delete(self.x_train, is_deviate, 0), np.delete(self.r_train, is_deviate, 0)     
    
    def gradient(self):
        
        if self.scaler is not None:
            scaler = self.scaler
            self.x_train = scaler.fit_transform(self.x_train)
            self.x_test = scaler.transform(self.x_test)
            if self.x_val is not None:
                self.x_val = scaler.transform(self.x_val)
        x_train_nn = torch.Tensor(self.x_train).to(self.device)   
        x_test_nn = torch.Tensor(self.x_test).to(self.device)
        r_train_nn = torch.Tensor(self.r_train).to(self.device)
        r_test_nn = torch.Tensor(self.r_test).to(self.device)
        config_nn = {
            "n_inputs" : self.num_features,
            "n_outputs": self.num_class,
            "n_epochs": self.n_epochs,
            "n_steps": self.n_steps,
            "batchnorm": self.batchnorm,
            "hidden_size": self.hidden_size,
            "max_batch": self.max_batch
            }        
        if self.bayes == True:
            config_nn["batches"], config_nn["lr"], config_nn["layers"]  = self.batches_bayes, self.lr_bayes, self.layers_bayes
            config_nn["rep"], config_nn["set"], config_nn["n_val_splits"]  = self.rep, self.set, self.n_val_splits
            config_nn["cval"], config_nn["val_sets"], config_nn["n_trials"] = self.cval, self.val_sets, self.n_trials      
            bayes = cslr_bayes(x_train_nn, r_train_nn, config_nn)
            best_params_bayes = bayes.bayes()
            best_lr = best_params_bayes.get("lr_rate", ""), 
            best_batch_size = best_params_bayes.get("batch_size" "")
            best_layer = best_params_bayes.get("num_ayer" "")
            config_nn["dnn_layers"] = best_layer         
            config_nn["batch_size"] = (2**best_batch_size if best_batch_size<self.max_batchsize else len(x_train_nn))
        else:
            best_lr = self.lr_rate
            config_nn["batch_size"] = self.batch_size
            config_nn["dnn_layers"] = self.numlayer
        self.model = LinearNet(config_nn).to(self.device)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=best_lr)
        optimization = Optimization(self.model, optimizer, config_nn)  
        optimization.train(x_train_nn, r_train_nn, x_test_nn, r_test_nn)
        _, _, test_probs = optimization.evaluate(x_test_nn, r_test_nn)
        _, _, train_probs = optimization.evaluate(x_train_nn, r_train_nn)
        if self.x_val is not None:
            x_val_nn = torch.Tensor(self.x_val).to(self.device)
            r_val_nn = torch.Tensor(self.r_val).to(self.device)                
            _, _, val_probs = optimization.evaluate(x_val_nn, r_val_nn)
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if "layer" in name:
                    self.beta_initial = param.data        
        return((test_probs, train_probs, val_probs) if self.x_val is not None else (test_probs, train_probs))
    
    def ret_model(self):
        return self.model
    
    def mip_opt(self):
    
        if self.give_initial == True:
            if self.beta_initial is None:
                self.gradient()
            x_train, r_train = self.opt_initial(self.beta_initial)
            model = Optimization_MIP({}, x_train, r_train)
            for i in range(self.num_class):
                for j in range(self.num_features):
                    model.beta[i,j].start = self.beta_initial[i,j]
            model.m.update() 
        else:
            model = Optimization_MIP({}, self.x_train, self.r_train)
        model.m.modelSense = GRB.MAXIMIZE
        model.m.setParam(GRB.Param.TimeLimit, self.time_limit)
        model.m.optimize()
        test_probs = np.zeros((len(self.x_test), self.num_class))
        train_probs = np.zeros((len(self.x_train), self.num_class))        
        try:
            for i in range(len(self.x_test)):
                for k in range(self.num_class):
                    test_probs[i,k] = sum(self.x_test[i,j] * model.beta[k,j].x for j in range(self.num_features))
            for i in range(len(self.x_train)):
                for k in range(self.num_class):
                    train_probs[i,k] = sum(self.x_train[i,j] * model.beta[k,j].x for j in range(self.num_features))  
            if self.x_val is not None:
                val_probs = np.zeros((len(self.x_val), self.num_class))       
                for i in range(len(self.x_val)):
                    for k in range(self.num_class):
                        val_probs[i,k] = sum(self.x_val[i,j] * model.beta[k,j].x for j in range(self.num_features))  
            return((test_probs, train_probs, val_probs) if self.x_val is not None else (test_probs, train_probs))
        except:
             return((test_probs, train_probs, val_probs) if self.x_val is not None else (test_probs, train_probs))

